# Проведение A/B-тестирование на основе данных


В рамках данной презентации рассмотрены конверсии, а также оцениваются статистические значимости их различия для двух целевых групп - контрольной и оцениваемой (новой), указанных в поле variation.

Сначала подключим ключевые библиотеки для работы.

In [1]:
#!pip install psycopg2-binary
import psycopg2;  #for working with Postgre
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import ttest_ind_from_stats as ttest

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Создадим подключение к имеющейся БД.

In [0]:
conn = psycopg2.connect(dbname='db_market', user='market_user', 
                        password='yiVuQH6LfI1ViMVCTcqi', host='40.85.140.102')

Напишем функцию для обработки запросов системой Postgre.

In [0]:
def perform_query(connection, query):
  '''
  connection - psycopg2.connect to neccesary DB
  query - SQL-like query

  return: query's answer as DataFrame
  '''
  with conn:
    return pd.read_sql_query(query, conn) 

Для проведения сегментации, целесообразно также реализовать функцию, конструирующую запрос. В качестве исходного запроса будем использовать:

In [0]:
core_query ='SELECT sum(loads) as sessions, sum(s1) as short, sum(s2) as entire \
              FROM \
              ( \
              SELECT variation, max(loads) as loads, max(submit_short) as s1, max(submit) as s2 \
              FROM abtest \
              {} \
              GROUP BY  session_id, variation \
              ) \
              AS sessions_data \
              GROUP BY variation \
              HAVING max(loads) > 0'

в котором поле 6 строки {} будет настраиваться в зависимости от потребностей с помощью функции create_query():

In [0]:
def create_query(segment_feature, core_query = core_query):
  return core_query.format(segment_feature)

#1. Об оценивании статистической значимости

Поскольку в имеющихся данных рассматривается две вариации, а также данные доступны по итогам проведения тестирования (не real-time), целесообразно проводить оценивание значимости изменений с помощью классической статистики, а именно проверки гипотезы о равенстве средних двух выборочных распределений.

Будем считать значимым абсолютное отклонение для короткой заявки в 3%, для полной заявки - 1%. Кроме того, положим уровень значимости равным 0.05, а ошибку второго рода - 0.2. Эти параметры могут быть выбраны произвольно, поэтому глобально по тексту будем придерживаться этих значений.

В списке параметров присутствует также величина delta_ste, когда определяет величину значимого отклонения конверсии частичной в полную заявку. Однако во-первых, данная величина коррелирована с двумя другими конверсиями, а во-вторых, объема данных как правило либо не хватает для оценивания изменения абсолютного отклонения на 5%, либо данных достаточно, и гипотеза выполняется. В дальнейшем величина этой конверсии для каждой из вариаций выводится в таблице, но стат. значимость не проверяется (при желании, это можно сделать, раскомментировав в функции variation_importance выполнение функции perform_stat_test - см. далее).

In [0]:
delta_short = 0.03
delta_entire = 0.01
#delta_ste = 0.05 #short to entire
alpha = 0.05
beta = 0.2

Также введем еще несколько функций, предназначенных для оценивания влияния внесенных правок.

In [0]:
def perform_stat_test(a_c, n_c, a_t, n_t, delta,
                      alpha = alpha, beta = beta):
  '''
  perform mean test for given success/total counts.
  '''
  #define basic characteristics
  p_c = a_c/n_c
  p_t = a_t/n_t
  std_c = np.sqrt(p_c*(1-p_c))
  std_t = np.sqrt(p_t*(1-p_t))

  #define required numbers of observation.
  n_required = np.ceil(2*p_c*(1-p_c)/delta**2 * (norm.ppf(1-alpha/2) + norm.ppf(1-beta))**2)

  #perform stat test
  Z, Pv = ttest(p_c,std_c,n_c,p_t,std_t,n_t, equal_var = False)
  if min(n_c, n_t) < n_required:
    print('Недостаточно данных для наблюдений. Требуемый объем выборки вариации: ', n_required)
  if Pv < alpha:
    print('Есть основания считать вариации различными. P-value: ',Pv)
  else:
    print('Нет оснований считать вариации различными. P-value: ',Pv)


def variation_importance(query, conn = conn):
  '''
  return text fields and df with basic stats for given query.
  '''
  df = perform_query(conn, query)
  #transform data fields
  df = df.rename({0:"control",1:"test"})
  df = df.assign(session_to_short = lambda x: x.short/x.sessions)
  df = df.assign(session_to_entire = lambda x: x.entire/x.sessions)
  df = df.assign(short_to_entire = lambda x: x.entire/x.short)


  #perform tests for short and entire' submits
  print('Конверсия сессий в короткую заявку по разным вариациям: ')
  perform_stat_test(df['short']['control'], df['sessions']['control'],
                    df['short']['test'],    df['sessions']['test'],
                    delta = delta_short)
  print('\n')
  #print('Конверсия коротких заявок по разным вариациям в полные: ')
  #perform_stat_test(df['entire']['control'], df['short']['control'],
  #                  df['entire']['test'],    df['short']['test'],
  #                  delta = delta_ste)
  print('\n')
  print('Конверсия сессий в полную заявку по разным вариациям: ')
  perform_stat_test(df['entire']['control'], df['sessions']['control'],
                    df['entire']['test'],    df['sessions']['test'],
                    delta = delta_entire)
  print('\n') 
  #show pivot
  return df[['sessions','short','session_to_short','entire','short_to_entire','session_to_entire']]

# 2. Оценивание значимости изменений на всей выборке

Теперь можно (наконец-то) начать проверять эффективность внесенных правок. Рассмотрим конверсию по всем данным с помощью описанных ранее функций:


In [8]:
segment_feature = ''
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.4350100977239145




Конверсия сессий в полную заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.3426980968943041




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,28448,5001,0.175794,2713,0.542492,0.095367
test,28532,5087,0.178291,2788,0.548064,0.097715


В таблице: строки control и test соответствуют контрольной и тестовым вариациям соответственно; sessions - количество сессий в вариации, short - количество коротких заявкок (submit_short); session_to_short - конверсия сессий в короткие заявки; entire - количество полных заявок (submit); short_to_entire, session_to_entire - доля конверсий коротких заявок и сессий в полные заявки соответственно.

Данная форма будет типовой в рамках отчета. Были проверены гипотезы о равенстве средних для конверсий коротких и полной заявок, и общий тренд в сторону улучшения на данном объеме не замечен. Отклонения будут пологаться значимыми, если P-значение меньше заданного уровня значимости на достаточном объеме наблюдений (если это не так, это будет написано).

 Изучим отдельно распределение по сегментам.
 

# 2.1 Оценивание значимости для разных сегментов используемых устройств


Посмотрим, какие сегменты наблюдаются в генеральной совокупности:

In [9]:
query = 'SELECT device_type, count(device_type) \
         FROM abtest  \
         GROUP BY device_type'
perform_query(conn, query)

,device_type,count
0,HIGHEND_PHONE,45219
1,SET_TOP_BOX,4
2,TABLET,872
3,UNKNOWN,4
4,BOT,20
5,CONNECTED_TV,3
6,PERSONAL_COMPUTER,33878


Исходя из увиденного, целесообразно оценивать значимость изменений в сегментах мобильных устройств и ПК.

In [10]:
segment_feature = ' WHERE device_type = \'HIGHEND_PHONE\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.4787146422844515




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  18510.0
Нет оснований считать вариации различными. P-value:  0.2730173185372016




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,14499,3755,0.258983,1980,0.527297,0.136561
test,14438,3792,0.262640,2036,0.536920,0.141017


In [11]:
segment_feature = ' WHERE device_type = \'PERSONAL_COMPUTER\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.30705790258397303




Конверсия сессий в полную заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.48358927124270235




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,13765,1236,0.089793,724,0.585761,0.052597
test,13662,1179,0.086298,693,0.587786,0.050725


Исходя из рассчитанного, можно сделать вывод о том, оснований для принятия соответствующих альтернатив на данном объеме нет.

# 2.2 Оценивание значимости для разных сегментов пользователей

Покажем, какие виды пользователей встретились в выборочной совокупности.

In [12]:
query = 'SELECT user_type, count(user_type) \
FROM abtest \
GROUP BY user_type \
ORDER BY count(user_type) DESC '
perform_query(conn, query)

,user_type,count
0,Visitor,57106
1,Client-Heavy,9866
2,Client-Light,3885
3,Applicant,1087
4,Applicant: Credit Cards;Tinkoff Platinum,29
5,Applicant: Loans;Cash Loan,21
6,Applicant: Debit Cards;Tinkoff Black,7
7,Applicant: Loans;Cash Loan Realty,5
8,Applicant: Credit Cards;Cash Loan,5
9,Applicant: Loans;Car Loan,3


Исходя из распределения, кажется целесообразным рассмотреть сегменты обычных посетителей, а также Light и Heavy клиентов.

In [13]:
segment_feature = ' WHERE user_type = \'Visitor\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.15296213041174883




Конверсия сессий в полную заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.36639978189030376




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,19787,3924,0.198312,2121,0.540520,0.107192
test,19797,4040,0.204071,2178,0.539109,0.110017


In [14]:
segment_feature = ' WHERE user_type = \'Client-Heavy\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.8059837791148893




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  4647.0
Нет оснований считать вариации различными. P-value:  0.22512979763458296




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,4160,228,0.054808,127,0.557018,0.030529
test,4050,217,0.053580,143,0.658986,0.035309


In [15]:
segment_feature = ' WHERE user_type = \'Client-Light\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  3290.0
Есть основания считать вариации различными. P-value:  0.029359322961688052




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  19131.0
Есть основания считать вариации различными. P-value:  0.037908925239833746




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,1225,309,0.252245,174,0.563107,0.142041
test,1186,346,0.291737,205,0.592486,0.172850


В сегментах посетителей и Heavy клиентов значимых отклонений выявлено не было. Кроме того, для категории Heavy клиентов уже не хватает данных для принятия взшеннего решения (иначе говоря, ошибки 1го и 2го рода сейчас существенно выше заложенных).

Категория Light слиентов является крайне сложной с точки зрения принятия статистического решения (но выглядит позитивно). Статистика равенства средних показывает неоднородность выборочных совокупностей, в то же время их объем не позволяет сделать вывод об отклонении основной гипотезы при заданной мощности.
Вообще говоря, можно используя минимальный объем вариации (1186 здесь) вычислить какую абсолютную разницу стоит рассматривать на подобном объеме выборки. Иначе говоря, если, скажем, рассматривать в качестве альтернативы гипотезу об отличии конверсии в тестовой вариации на 6%, то данная альтернатива может быть принята. Для подобных рассуждений требуется логическое обоснование с точки зрения продукта (а могло ли изменения привести к таким масштабам). Поскольку в данном задаче подобная задача не стоит, будем считать подобное отклонение незначимым на имеющемся объеме, а подобный сегмент - целесообразным для дальнейшего проведения тестирования. Кроме того, возможен анализ выборок другими методами - например, при наличии хронологии (вероятно, можно было бы положить, что записи делались в порядке записи в БД), с помощью "бандитов".

# 2.3 Оценивание значимости для разных сегментов трафика


Для источников трафика выделить явным образом не представляется возможным из-за относительной произвольности заполнения поля. В то же время, можно выделить 3 ключевых источника, обеспечивающих около 80% записей в таблице: tinkoff, yandex, google. Поэтому рассмотрим именно эти сегменты.

In [16]:
segment_feature = ' WHERE source LIKE \'%tinkoff%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Есть основания считать вариации различными. P-value:  0.02939590611233198




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  14785.0
Нет оснований считать вариации различными. P-value:  0.16856203122419244




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,2945,583,0.197963,310,0.531732,0.105263
test,2917,645,0.221118,340,0.527132,0.116558


Данные показывают довольно оптимистичную картину для канала Tinkoff. На имеющихся данных замечен значимый рост конверсии в неполную заявку на приблизительно 2,2%. В имеющихся данных также наблюдается рост конверсии в полную заявку, однако на данном объеме его можно считать в рамках статистического разброса. 

In [17]:
segment_feature = ' WHERE source LIKE \'%yandex%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.7971650976982966




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  15001.0
Нет оснований считать вариации различными. P-value:  0.5532329007906513




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,8186,1552,0.189592,876,0.564433,0.107012
test,8318,1564,0.188026,914,0.584399,0.109882


Для сегмента с канала Yandex конверсия в неполную заявку практически не изменилась. Для полных заявок сложно сделать какой-либо вывод относительно смещения конверсии в них сессий из-за недобора данных. 

In [18]:
segment_feature = ' WHERE source LIKE \'%google%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Есть основания считать вариации различными. P-value:  0.02318022519241565




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  11798.0
Нет оснований считать вариации различными. P-value:  0.3004979086178898




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,11753,1759,0.149664,962,0.546902,0.081851
test,11678,1626,0.139236,913,0.561501,0.078181


В случае с трафиком Google, наблюдается статистически значимое (на уровне 0,05) падение конверсии в неполную заявку. Для анализа конверсии в полную заявку наблюдается минимальный недобор данных, однако, данные скорее говорят о том, что на полную конверсию изменение не повлияло.

# 2.4 Оценивание значимости для различных каналов привлечения

Для сегментации по полю medium достаточно сложно выделить несколько кластеров - наиболее численных является сегмент привлечения CONTEXT, сессий по которым существенно больше чем за следующими за ним AFFILAT, TARGET и MAILING.

In [19]:
segment_feature = ' WHERE medium LIKE \'%ctx%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Нет оснований считать вариации различными. P-value:  0.12596175126131773




Конверсия сессий в полную заявку по разным вариациям: 
Есть основания считать вариации различными. P-value:  0.01723144492150098




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,11564,1054,0.091145,602,0.571157,0.052058
test,11691,999,0.085450,530,0.530531,0.045334


In [20]:
segment_feature = ' WHERE medium LIKE \'%aft%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  2751.0
Нет оснований считать вариации различными. P-value:  0.48522706040641683




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  13989.0
Нет оснований считать вариации различными. P-value:  0.9298146190304188




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,1891,371,0.196192,187,0.504043,0.098889
test,1826,375,0.205367,179,0.477333,0.098028


In [21]:
segment_feature = ' WHERE medium LIKE \'%trg%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  3335.0
Нет оснований считать вариации различными. P-value:  0.6823005613172961




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  17147.0
Нет оснований считать вариации различными. P-value:  0.850155462530568




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,633,163,0.257504,79,0.484663,0.124803
test,639,171,0.267606,82,0.479532,0.128326


In [22]:
segment_feature = ' WHERE medium LIKE \'%mln%\' '
variation_importance(create_query(segment_feature))

Конверсия сессий в короткую заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  2410.0
Нет оснований считать вариации различными. P-value:  0.2555639585596363




Конверсия сессий в полную заявку по разным вариациям: 
Недостаточно данных для наблюдений. Требуемый объем выборки вариации:  14214.0
Нет оснований считать вариации различными. P-value:  0.9425875833942192




,sessions,short,session_to_short,entire,short_to_entire,session_to_entire
control,586,97,0.165529,59,0.608247,0.100683
test,559,79,0.141324,57,0.721519,0.101968


Исходя из полученного, можно сразу выбросить каналы AFFILAT, TARGET и MAILING из рассмотрения - данных по ним объективно мало для принятия взвешенного решения.

По каналу CONTEXT отмечается негативная тенденция в количестве конверсий сессий в полные заявки. Для конверсии в частичную заявку также наблюдается падение, однако оно может являться лишь результатом стастического разброса.

# 3. Выводы

В рамках работы проведен анализ эффективности изменений в продукте путем сравнения долей конверсий в вариациях методами математической статистики. Исходя из результатов работы, можно сделать следующие выводы:
1. Явного преобладания у новой вариации относительно тестовой не наблюдается;
2. Сегмент клиентов, пришедших через Google, воспринял изменения негативно (на уровне значимости 0,05) на этапе короткой заявки, однако на итоговой заявке это не сказалось (вообще говоря, количества наблюдений недостаточно, но объем выборки немногим отличается от требуемого). По необходимости имеет смысл недолго продлить тест (не хватает порядка 250 сессий) для полноценного анализа итоговой конверсии в полную заявку;
3. Для клиентов, пришел с Tinkoff, замечен значимый (на уровне значимости 0,05) рост конверсии в короткую заявку. В связи с существенным недобором статистических данных, оценить значимость роста конверсии в полную заявку не представляется возможным - целесообразно по мере возможности продлить тестирование;
4. Канал привлечения CONTEXT отреагировал на изменение отрицательно (при уровне значимости 0,05) - наблюдается сокращение доли конверсий сессий в полные заявки. Оснований же для принятия вывода о сокращении числа конверсий в короткие заявки на основании имеющихся данных нет;
5. У сегмента пользователей Light -Clients целесообразно продолжение тестирования, поскольку имеющийся объем не позволяет сделать обоснованного вывода об эффективности сделанных изменений.